In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [3]:
# start = 1719503720
# end = 1719506560


importlib.reload(overview)
anubis_file = "proAnubis_240627_1720.raw"
data, times, fReader = overview.get_chunks(anubis_file, start="2024-06-27 16:55:20", end = "2024-06-27 17:55:20", tdc5 = True)

Initial time 2024-06-27 16:20:18.087426 1719501618.087426
Start: 1719503720.0
Bigger than start: True


Skipping Events proAnubis_240627_1720.raw:   1%|          | 24/2102 [00:05<07:23,  4.68Events/s]


KeyboardInterrupt: 

In [13]:
storage = "final.pkl"
with open(storage, "wb") as f:
    pickle.dump(data, f)
print("Saved")


Saved


In [31]:
storage = "tdc_onljkjy.pkl"
with open(storage, "wb") as f:
    pickle.dump(tdc5Reads, f)
print("Saved")


Saved


In [2]:
importlib.reload(AtTools)
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data("tdc_only.pkl", trigger_channel=4, bcr_channel=2, amount_of_events=784_480 )
#analyser.correctionBCR()

Reading:  11%|█         | 84901/784480 [00:24<01:30, 7757.64Events/s]

In [3]:
storage = "bcr.pkl"
with open(storage, "wb") as f:
    pickle.dump(analyser.anubis_data, f)
print("Saved")


Saved


In [2]:
storage_name = "bcr.pkl"
with open(storage_name, "rb") as inp:
    anubis_data = pickle.load(inp)
print("Chunks Loaded")


In [3]:
importlib.reload(AtTools)
atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas.root"
analyser.getAtlasData(atlas_file)
print("Loading done")
with tqdm(total=len(analyser.atlas_data)) as pbar:
    for i in range(len(analyser.atlas_data)):
        hits = analyser.atlas_data.iloc[i]
        for p in range(len(hits["mu_eta"])):
            has_our_hit = False
            if 0.93 > hits["mu_eta"][p] > 0.83 and abs(hits["mu_phi"][p] - 1.57) < 0.1:
                has_our_hit = True
                break
        if not has_our_hit:
            analyser.atlas_data.drop(i, inplace=True)
        pbar.update(1)

In [47]:
muons = 0

for i in range(len(analyser.atlas_data.values)):
    hits = analyser.atlas_data.iloc[i]
    for p in range(len(hits["mu_eta"])):
        if 0.93 > hits["mu_eta"][p] > 0.83 and abs(hits["mu_phi"][p] - 1.57) < 0.1:
            muons += 1
            print(hits["mu_eta"][p], hits["mu_phi"][p])

print(muons)
print(len(analyser.atlas_data.values)/(analyser.atlas_data.iloc[-1]["TimeStamp"]-analyser.atlas_data.iloc[0]["TimeStamp"]))

0.9055982 1.6272389
0.8418124 1.5141773
0.9003911 1.5831764
0.91317505 1.6379361
0.88016343 1.6324259
0.8405359 1.6310602
0.84422237 1.5252458
0.9092756 1.6565939
0.88896704 1.6592686
0.8551121 1.6690565
0.92434996 1.6119776
0.916709 1.501556
0.884171 1.6669021
0.8757307 1.5745344
0.86695725 1.6475704
0.8437779 1.5699189
0.83327657 1.63008
0.92696124 1.5589747
0.855018 1.644672
0.8809462 1.591635
0.9050021 1.4820646
0.84083986 1.6163306
0.89324814 1.5247784
0.89414024 1.6552016
0.91686696 1.4933925
0.8306013 1.5401787
0.8653364 1.6211209
0.8304299 1.5747046
0.89442986 1.5560509
0.9243073 1.5601153
0.8524486 1.6571232
0.83291835 1.5954986
0.92991513 1.4866681
0.8849151 1.6025742
0.84156954 1.6679847
0.87084275 1.5110704
0.90370184 1.6099197
0.859081 1.4824331
0.87210983 1.638158
0.8932732 1.6309835
0.8607471 1.5688298
0.8845042 1.4730979
0.84206337 1.6685599
0.85707885 1.5931025
0.8783002 1.6647258
0.8830836 1.4703554
0.8715052 1.5291833
0.84967834 1.668716
0.87012815 1.6295933
0.860924

In [42]:
#plot density by eta
hist_eta = []
for i in range(len(analyser.atlas_data.values)):
    hits = analyser.atlas_data.iloc[i]
    for p in range(len(hits["mu_eta"])):
            hist_eta.append(hits["mu_eta"][p])
plt.title("Density of muons by eta")
plt.xlabel("Eta")
plt.xlim(-3, 3)
plt.hist(hist_eta, bins=100, density=True)
plt.show()


In [ ]:
hist, bins = np.histogram(hist_eta, bins=100, density=True)

In [11]:
# set the times
analyser.atlas_data = analyser.atlas_data[analyser.atlas_data["TimeStamp"] > analyser.anubis_data[0].timeStamp]
print("Anubis", (round(analyser.anubis_data[0].timeStamp), round(analyser.anubis_data[-1].timeStamp)))
print("ATLAS",(analyser.atlas_data.iloc[0]["TimeStamp"], analyser.atlas_data.iloc[-1]["TimeStamp"]))

Anubis (1719503720, 1719506762)
ATLAS (np.uint32(1719503721), np.uint32(1719506684))


In [12]:
times = analyser.atlas_data["TimeStamp"]
print(times) 
#1719504220
#1719503530

hist, bins = np.histogram(times, bins=100)
plt.plot(bins[:-1], hist)
plt.show()

9378      1719503721
9380      1719503721
10762     1719503721
10919     1719503721
9367      1719503721
             ...    
144140    1719506684
145102    1719506684
145264    1719506684
144324    1719506684
144329    1719506684
Name: TimeStamp, Length: 135594, dtype: uint32


In [79]:
#Calculate the BCR offset
importlib.reload(AtTools)
anubis = AtTools.BCRHistogram(analyser.anubis_data, atlas=False, plot = False)
atlas = AtTools.BCRHistogram(analyser.atlas_data, atlas=True, plot = False)

window_size = 100
anubis = np.convolve(np.append(anubis, anubis[:window_size]), np.ones(window_size)/window_size, mode='valid')
atlas = np.convolve(np.append(atlas, atlas[:window_size]), np.ones(window_size)/window_size, mode='valid')
error = 100
errors = []
for offset in range(-200,200):
    errors.append(np.sum((atlas - np.roll(anubis, offset))**2))
    if np.sum((atlas - np.roll(anubis, offset))**2) < error:
        error = np.sum((atlas - np.roll(anubis, offset))**2)
        best_offset = offset

print("Best offset", best_offset)
plt.plot(atlas, label="Atlas")
#plt.plot(anubis, label="Original Anubis")
plt.plot(np.roll(anubis, best_offset), label="Offset Anubis")
plt.xlim(0, 3565)
plt.ylabel("Counts (normalized)")
plt.xlabel("BX")
plt.title("Anubis and Atlas BCR Histograms aligned - smoothed")
plt.legend()
plt.show()

plt.plot(list(range(-200,200)), errors, label="Errors")
plt.xlabel("Offset")
plt.ylabel("Sum of Square Errors")
plt.title("Sum of Square Errors vs Offset")
plt.vlines(best_offset,0, max(errors), color = "red", label=f"Best Offset: {best_offset}")
plt.ylim(0,max(errors))
plt.legend(loc = "lower left") #position

plt.show()
    

Best offset 114


In [24]:
importlib.reload(AtTools)
analyser.pairBCRwithEvents()

Matching:   7%|▋         | 9773/135594 [08:33<1:50:04, 19.05Events/s] 


KeyboardInterrupt: 

In [48]:
#difference
best_offset = 114
eta = []
phi = []
times = []
matches = analyser.matches
for i in range(len(matches)):
    atlas_hits = matches[i][0]
    for p in range(len(matches[i][0]["mu_eta"])):    
        if 0.93 > atlas_hits["mu_eta"][p] > 0.83 and abs(atlas_hits["mu_phi"][p] - 1.57) < 0.1:        
            if matches[i][1]:
                for trigger in matches[i][1]:
                    #if abs(round(trigger.bcId-matches[i][0]["BCID"])) > 1000:
                    #    continue
                    eta.extend(matches[i][0]["mu_eta"])
                    phi.extend(matches[i][0]["mu_phi"])       
                    for _ in range(len(matches[i][0]["mu_eta"])):
                        times.append(round(trigger.bcId+best_offset-matches[i][0]["BCID"]))

In [86]:
#3D plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(eta, phi, times)
ax.set_xlabel('eta')
plt.ylim(-3.2,3.2)
plt.xlim(-3.2,3.2)
ax.set_ylabel('phi')
ax.set_zlabel('time')
plt.show()

In [50]:
#Histograms
plt.hist2d(eta, phi, bins=30, cmap='plasma')
plt.xlim(-np.pi, np.pi)
plt.ylim(-np.pi, np.pi)
plt.xlabel("eta")
plt.ylabel("phi")
# Add a color bar to show the color scale
plt.colorbar()
plt.show()

plt.hist(times, bins=range(min(times), max(times) + 2, 1))
#plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
plt.xlim(-500,500)
plt.ylabel("Frequency")
plt.title("Time difference between BCR and trigger")
plt.show()

In [21]:

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

hist, xedges, yedges = np.histogram2d(eta, phi, bins=30, range=[[-np.pi, np.pi], [-np.pi, np.pi]])

# Prepare the grid for the surface plot
xpos, ypos = np.meshgrid(xedges[:-1] + 0.5 * (xedges[1] - xedges[0]),
                         yedges[:-1] + 0.5 * (yedges[1] - yedges[0]),
                         indexing="ij")

# Initialize the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.plot_surface(xpos, ypos, hist, cmap='plasma')

# Set labels
ax.set_xlabel('eta')
ax.set_ylabel('phi')
ax.set_zlabel('Count')

# Set limits
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)

# Show the plot
plt.show()
